In [289]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [290]:
kospi1 = pd.read_csv("./코스피200연결.csv", encoding = "euc-kr")
kospi2 = pd.read_csv("./코스피200개별.csv", encoding = "euc-kr")
st13 = pd.read_csv("./코스피주식(2013).csv", encoding = "euc-kr")
st14 = pd.read_csv("./코스피주식(2014).csv", encoding = "euc-kr")
st15 = pd.read_csv("./코스피주식(2015).csv", encoding = "euc-kr")
st16 = pd.read_csv("./코스피주식(2016).csv", encoding = "euc-kr")
st17 = pd.read_csv("./코스피주식(2017).csv", encoding = "euc-kr")
st18 = pd.read_csv("./코스피주식(2018).csv", encoding = "euc-kr")
st19 = pd.read_csv("./코스피주식(2019).csv", encoding = "euc-kr")
st20 = pd.read_csv("./코스피주식(2020).csv", encoding = "euc-kr")
st21 = pd.read_csv("./코스피주식(2021).csv", encoding = "euc-kr")
st22 = pd.read_csv("./코스피주식(2022).csv", encoding = "euc-kr")


In [291]:
kospi1

,회사명,거래소코드,회계년도,거래월(*),종가,자산(*)(IFRS연결),부채(*)(IFRS연결),당기순이익(손실)(IFRS연결),영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결),매출액(수익)(*)(IFRS연결),비유동자산(*)(IFRS연결),매출총이익(손실)(IFRS연결),법인세비용(IFRS연결),이자비용(IFRS연결),감가상각비(IFRS연결),유동자산(*)(IFRS연결),유동부채(*)(IFRS연결),현금및현금성자산(*)(IFRS연결)
0,(주)BNK금융지주,138930,2013/12,1.0,14400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,(주)BNK금융지주,138930,2013/12,2.0,15650.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,(주)BNK금융지주,138930,2013/12,3.0,15300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,(주)BNK금융지주,138930,2013/12,4.0,14500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,(주)BNK금융지주,138930,2013/12,5.0,15700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20812,효성티앤씨(주),298020,2022/12,8.0,313500.0,4.319879e+09,2.804356e+09,19183248.0,293383061.0,8.882730e+09,2.176811e+09,432144813.0,11475356.0,0.0,14803000.0,2.143068e+09,2.184964e+09,103107215.0
20813,효성티앤씨(주),298020,2022/12,9.0,259000.0,4.319879e+09,2.804356e+09,19183248.0,293383061.0,8.882730e+09,2.176811e+09,432144813.0,11475356.0,0.0,14803000.0,2.143068e+09,2.184964e+09,103107215.0
20814,효성티앤씨(주),298020,2022/12,10.0,261500.0,4.319879e+09,2.804356e+09,19183248.0,293383061.0,8.882730e+09,2.176811e+09,432144813.0,11475356.0,0.0,14803000.0,2.143068e+09,2.184964e+09,103107215.0
20815,효성티앤씨(주),298020,2022/12,11.0,357000.0,4.319879e+09,2.804356e+09,19183248.0,293383061.0,8.882730e+09,2.176811e+09,432144813.0,11475356.0,0.0,14803000.0,2.143068e+09,2.184964e+09,103107215.0


In [292]:
kospi1.columns = ['회사명', '종목코드', '회계년도', '거래월', '종가', '자산', '부채', '당기순이익', '현금흐름', '매출액', '비유동자산', '매출총이익', '법인세비용', '이자비용', '감가상각비', '유동자산',
       '유동부채', '현금및현금성자산']
kospi2.columns = ['회사명', '종목코드', '회계년도', '거래월', '종가', '자산', '부채', '당기순이익', '현금흐름', '매출액', '비유동자산', '매출총이익', '법인세비용', '이자비용', '감가상각비', '유동자산',
       '유동부채', '현금및현금성자산']

In [293]:
st13['회계년도'] = '2013/12'
st14['회계년도'] = '2014/12'
st15['회계년도'] = '2015/12'
st16['회계년도'] = '2016/12'
st17['회계년도'] = '2017/12'
st18['회계년도'] = '2018/12'
st19['회계년도'] = '2019/12'
st20['회계년도'] = '2020/12'
st21['회계년도'] = '2021/12'
st22['회계년도'] = '2022/12'


In [294]:
df = pd.concat([st13, st14, st15, st16, st17, st18, st19, st20, st21, st22], axis=0)

In [295]:
df['종목코드'] = df['종목코드'].apply(lambda x: x.lstrip('0'))

In [296]:
kospi = kospi1.fillna(kospi2)

In [297]:
kospi['종목코드'] = kospi['종목코드'].astype(str)

In [298]:
kospi = pd.merge(kospi, df[['종목코드', '회계년도', '상장주식수', '시가총액']], on=['종목코드', '회계년도'], how='left')

In [299]:
kospi

,회사명,종목코드,회계년도,거래월,종가,자산,부채,당기순이익,현금흐름,매출액,비유동자산,매출총이익,법인세비용,이자비용,감가상각비,유동자산,유동부채,현금및현금성자산,상장주식수,시가총액
0,(주)BNK금융지주,138930,2013/12,1.0,14400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,193379899.0,3.094078e+12
1,(주)BNK금융지주,138930,2013/12,2.0,15650.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,193379899.0,3.094078e+12
2,(주)BNK금융지주,138930,2013/12,3.0,15300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,193379899.0,3.094078e+12
3,(주)BNK금융지주,138930,2013/12,4.0,14500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,193379899.0,3.094078e+12
4,(주)BNK금융지주,138930,2013/12,5.0,15700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,193379899.0,3.094078e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20812,효성티앤씨(주),298020,2022/12,8.0,313500.0,4.319879e+09,2.804356e+09,19183248.0,293383061.0,8.882730e+09,2.176811e+09,432144813.0,11475356.0,0.0,14803000.0,2.143068e+09,2.184964e+09,103107215.0,4327682.0,1.514689e+12
20813,효성티앤씨(주),298020,2022/12,9.0,259000.0,4.319879e+09,2.804356e+09,19183248.0,293383061.0,8.882730e+09,2.176811e+09,432144813.0,11475356.0,0.0,14803000.0,2.143068e+09,2.184964e+09,103107215.0,4327682.0,1.514689e+12
20814,효성티앤씨(주),298020,2022/12,10.0,261500.0,4.319879e+09,2.804356e+09,19183248.0,293383061.0,8.882730e+09,2.176811e+09,432144813.0,11475356.0,0.0,14803000.0,2.143068e+09,2.184964e+09,103107215.0,4327682.0,1.514689e+12
20815,효성티앤씨(주),298020,2022/12,11.0,357000.0,4.319879e+09,2.804356e+09,19183248.0,293383061.0,8.882730e+09,2.176811e+09,432144813.0,11475356.0,0.0,14803000.0,2.143068e+09,2.184964e+09,103107215.0,4327682.0,1.514689e+12


In [300]:
kospi['pbr'] = (kospi['종가']*kospi['상장주식수'])/(kospi['자산']-kospi['부채'])
kospi['per'] = kospi['종가']/(kospi['당기순이익']/kospi['상장주식수'])
kospi['pcr'] = kospi['종가']/(kospi['현금흐름']/kospi['상장주식수'])
kospi['psr'] = kospi['종가']/(kospi['매출액']/kospi['상장주식수'])
kospi['roe'] = kospi['당기순이익']/(kospi['자산']-kospi['부채'])
kospi['roa'] = kospi['당기순이익']/kospi['자산']
kospi['roic'] = kospi['당기순이익']/(kospi['자산']-kospi['비유동자산'])
kospi['gpa'] = kospi['매출총이익']/kospi['자산']

In [301]:
# 분자(EBIT)
magic_ebit = kospi['당기순이익'] + kospi['법인세비용'] + kospi['이자비용']

# 분모
magic_cap = kospi['시가총액']
magic_debt = kospi['부채']

## 분모: 여유자금
magic_excess_cash = kospi['유동부채'] - kospi['유동자산'] + kospi[
    '현금및현금성자산']
magic_excess_cash[magic_excess_cash < 0] = 0
magic_excess_cash_final = kospi['현금및현금성자산'] - magic_excess_cash

magic_ev = magic_cap + magic_debt - magic_excess_cash_final

# 이익수익률
magic_ey = magic_ebit / magic_ev
kospi['매직'] = magic_ey

In [302]:
kospi = kospi.dropna()

In [303]:
len(kospi['회사명'].unique())

179

In [304]:
kospi.reset_index(drop=True, inplace=True)

In [305]:
kospi

,회사명,종목코드,회계년도,거래월,종가,자산,부채,당기순이익,현금흐름,매출액,...,시가총액,pbr,per,pcr,psr,roe,roa,roic,gpa,매직
0,(주)DB하이텍,990,2013/12,1.0,6550.0,1.058392e+09,8.536351e+08,-83046768.0,28380938.0,4.937520e+08,...,3.340898e+11,1419.290470,-3499.345087,10239.594604,588.573379,-0.405587,-0.078465,-0.372473,0.084327,-0.000243
1,(주)DB하이텍,990,2013/12,2.0,6890.0,1.058392e+09,8.536351e+08,-83046768.0,28380938.0,4.937520e+08,...,3.340898e+11,1492.963563,-3680.990481,10771.115545,619.125280,-0.405587,-0.078465,-0.372473,0.084327,-0.000243
2,(주)DB하이텍,990,2013/12,3.0,6250.0,1.058392e+09,8.536351e+08,-83046768.0,28380938.0,4.937520e+08,...,3.340898e+11,1354.284799,-3339.069740,9770.605538,561.615820,-0.405587,-0.078465,-0.372473,0.084327,-0.000243
3,(주)DB하이텍,990,2013/12,4.0,5970.0,1.058392e+09,8.536351e+08,-83046768.0,28380938.0,4.937520e+08,...,3.340898e+11,1293.612840,-3189.479415,9332.882410,536.455431,-0.405587,-0.078465,-0.372473,0.084327,-0.000243
4,(주)DB하이텍,990,2013/12,5.0,8490.0,1.058392e+09,8.536351e+08,-83046768.0,28380938.0,4.937520e+08,...,3.340898e+11,1839.660471,-4535.792334,13272.390563,762.898930,-0.405587,-0.078465,-0.372473,0.084327,-0.000243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18340,효성티앤씨(주),298020,2022/12,8.0,313500.0,4.319879e+09,2.804356e+09,19183248.0,293383061.0,8.882730e+09,...,1.514689e+12,895.220804,70724.639905,4624.426176,152.737768,0.012658,0.004441,0.008951,0.100036,0.000020
18341,효성티앤씨(주),298020,2022/12,9.0,259000.0,4.319879e+09,2.804356e+09,19183248.0,293383061.0,8.882730e+09,...,1.514689e+12,739.592307,58429.606811,3820.498819,126.185270,0.012658,0.004441,0.008951,0.100036,0.000020
18342,효성티앤씨(주),298020,2022/12,10.0,261500.0,4.319879e+09,2.804356e+09,19183248.0,293383061.0,8.882730e+09,...,1.514689e+12,746.731229,58993.599155,3857.376221,127.403274,0.012658,0.004441,0.008951,0.100036,0.000020
18343,효성티앤씨(주),298020,2022/12,11.0,357000.0,4.319879e+09,2.804356e+09,19183248.0,293383061.0,8.882730e+09,...,1.514689e+12,1019.438044,80538.106686,5266.092966,173.931047,0.012658,0.004441,0.008951,0.100036,0.000020


In [306]:
kospi.to_csv('./kospi.csv')

In [307]:
# # 외국계 기업 종목코드 제거
# kospi['종목코드'] = pd.to_numeric(kospi['종목코드'], errors='coerce')
# kospi = kospi[kospi['종목코드'] < 900000]

In [308]:
# kospi = kospi.astype({"종목코드" : "str"})

In [309]:
# kospi.info()

In [310]:
# data = kospi['pbr'].sort_values(ascending=False)

In [311]:
kospi['종가'].shift(1)

0             NaN
1          6550.0
2          6890.0
3          6250.0
4          5970.0
           ...   
18340    344500.0
18341    313500.0
18342    259000.0
18343    261500.0
18344    357000.0
Name: 종가, Length: 18345, dtype: float64

In [312]:
# 종가의 변화율 수익률 컬럼 생성
# kospi['수익률'] = (kospi['종가'] / kospi['종가'].shift(1) - 1) * 100

In [313]:
kospi.sort_values(by=['회계년도', '거래월'],ascending=True, inplace=True)
kospi

/var/folders/xs/8nhrbkk97jzf2f4zjswkk0b80000gn/T/ipykernel_1426/3249123550.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kospi.sort_values(by=['회계년도', '거래월'],ascending=True, inplace=True)


,회사명,종목코드,회계년도,거래월,종가,자산,부채,당기순이익,현금흐름,매출액,...,시가총액,pbr,per,pcr,psr,roe,roa,roic,gpa,매직
0,(주)DB하이텍,990,2013/12,1.0,6550.0,1.058392e+09,8.536351e+08,-8.304677e+07,2.838094e+07,4.937520e+08,...,3.340898e+11,1419.290470,-3499.345087,10239.594604,588.573379,-0.405587,-0.078465,-0.372473,0.084327,-0.000243
120,(주)강원랜드,35250,2013/12,1.0,31250.0,3.102037e+09,5.843404e+08,2.975963e+08,4.244071e+08,1.361287e+09,...,6.621458e+12,2655.459320,22465.466033,15752.894331,4911.266029,0.118202,0.095936,0.275450,0.204767,0.000059
240,(주)녹십자,6280,2013/12,1.0,140500.0,1.104265e+09,2.648562e+08,7.202771e+07,2.968948e+07,8.881682e+08,...,1.454974e+12,1956.089859,22796.207662,55304.399589,1848.702280,0.085808,0.065227,0.128403,0.248938,0.000063
360,(주)녹십자홀딩스,5250,2013/12,1.0,14150.0,1.624124e+09,5.244572e+08,8.079098e+07,1.756831e+07,9.382984e+08,...,5.855012e+11,605.136903,8236.676258,37877.817915,709.208474,0.073469,0.049744,0.129527,0.184121,0.000186
480,(주)농심,4370,2013/12,1.0,279000.0,2.215391e+09,6.987179e+08,8.689654e+07,1.622214e+08,2.086651e+09,...,1.526743e+12,1118.933972,19529.628424,10461.361316,813.292117,0.057294,0.039224,0.094623,0.264565,0.000077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17996,현대위아(주),11210,2022/12,12.0,49450.0,7.546451e+09,3.812800e+09,4.348200e+07,4.239810e+08,8.207614e+09,...,1.344797e+12,360.182795,30927.667871,3171.832828,163.847478,0.011646,0.005762,0.010189,0.077788,0.000064
18116,현대자동차(주),5380,2022/12,12.0,151000.0,2.557425e+11,1.648459e+11,7.983614e+09,1.062731e+10,1.425275e+11,...,3.226390e+13,354.951844,4041.264550,3035.941664,226.369566,0.087832,0.031217,0.053684,0.110729,0.000337
18236,현대제철(주),4020,2022/12,12.0,30600.0,3.680096e+10,1.767241e+10,1.038193e+09,2.179617e+09,2.734060e+10,...,4.083441e+12,213.473709,3933.219831,1873.467538,149.354470,0.054275,0.028211,0.078704,0.077001,0.000329
18290,효성첨단소재(주),298050,2022/12,12.0,333500.0,2.934713e+09,2.135156e+09,1.602384e+08,3.504464e+08,3.841373e+09,...,1.494063e+12,1868.611875,9324.000542,4263.313338,388.939769,0.200409,0.054601,0.120365,0.177875,0.000142


In [314]:
kospi.reset_index(drop=True)

,회사명,종목코드,회계년도,거래월,종가,자산,부채,당기순이익,현금흐름,매출액,...,시가총액,pbr,per,pcr,psr,roe,roa,roic,gpa,매직
0,(주)DB하이텍,990,2013/12,1.0,6550.0,1.058392e+09,8.536351e+08,-8.304677e+07,2.838094e+07,4.937520e+08,...,3.340898e+11,1419.290470,-3499.345087,10239.594604,588.573379,-0.405587,-0.078465,-0.372473,0.084327,-0.000243
1,(주)강원랜드,35250,2013/12,1.0,31250.0,3.102037e+09,5.843404e+08,2.975963e+08,4.244071e+08,1.361287e+09,...,6.621458e+12,2655.459320,22465.466033,15752.894331,4911.266029,0.118202,0.095936,0.275450,0.204767,0.000059
2,(주)녹십자,6280,2013/12,1.0,140500.0,1.104265e+09,2.648562e+08,7.202771e+07,2.968948e+07,8.881682e+08,...,1.454974e+12,1956.089859,22796.207662,55304.399589,1848.702280,0.085808,0.065227,0.128403,0.248938,0.000063
3,(주)녹십자홀딩스,5250,2013/12,1.0,14150.0,1.624124e+09,5.244572e+08,8.079098e+07,1.756831e+07,9.382984e+08,...,5.855012e+11,605.136903,8236.676258,37877.817915,709.208474,0.073469,0.049744,0.129527,0.184121,0.000186
4,(주)농심,4370,2013/12,1.0,279000.0,2.215391e+09,6.987179e+08,8.689654e+07,1.622214e+08,2.086651e+09,...,1.526743e+12,1118.933972,19529.628424,10461.361316,813.292117,0.057294,0.039224,0.094623,0.264565,0.000077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18340,현대위아(주),11210,2022/12,12.0,49450.0,7.546451e+09,3.812800e+09,4.348200e+07,4.239810e+08,8.207614e+09,...,1.344797e+12,360.182795,30927.667871,3171.832828,163.847478,0.011646,0.005762,0.010189,0.077788,0.000064
18341,현대자동차(주),5380,2022/12,12.0,151000.0,2.557425e+11,1.648459e+11,7.983614e+09,1.062731e+10,1.425275e+11,...,3.226390e+13,354.951844,4041.264550,3035.941664,226.369566,0.087832,0.031217,0.053684,0.110729,0.000337
18342,현대제철(주),4020,2022/12,12.0,30600.0,3.680096e+10,1.767241e+10,1.038193e+09,2.179617e+09,2.734060e+10,...,4.083441e+12,213.473709,3933.219831,1873.467538,149.354470,0.054275,0.028211,0.078704,0.077001,0.000329
18343,효성첨단소재(주),298050,2022/12,12.0,333500.0,2.934713e+09,2.135156e+09,1.602384e+08,3.504464e+08,3.841373e+09,...,1.494063e+12,1868.611875,9324.000542,4263.313338,388.939769,0.200409,0.054601,0.120365,0.177875,0.000142


In [315]:
kospi.loc[kospi["회계년도"] == "2013/12"][kospi["거래월"] == 1.0].sort_values(by = ["pbr"], ascending=False)

/var/folders/xs/8nhrbkk97jzf2f4zjswkk0b80000gn/T/ipykernel_1426/472554982.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  kospi.loc[kospi["회계년도"] == "2013/12"][kospi["거래월"] == 1.0].sort_values(by = ["pbr"], ascending=False)


,회사명,종목코드,회계년도,거래월,종가,자산,부채,당기순이익,현금흐름,매출액,...,시가총액,pbr,per,pcr,psr,roe,roa,roic,gpa,매직
17757,현대엘리베이터(주),17800,2013/12,1.0,98300.0,1.204332e+09,1.044260e+09,-342662157.0,1.043526e+08,1.066213e+09,...,6.516341e+11,8371.705395,-3910.779176,12841.809847,1256.856018,-2.140675,-0.284525,-0.525345,0.155419,-0.000597
15767,한국전력기술(주),52690,2013/12,1.0,80100.0,7.605037e+08,3.784537e+08,34407214.0,3.926585e+07,7.554839e+08,...,2.312310e+12,8013.146009,88976.166452,77966.528166,4052.266631,0.090059,0.045243,0.071854,0.197899,0.000020
10835,삼성엔지니어링(주),28050,2013/12,1.0,155500.0,5.928776e+09,5.023117e+09,-708658933.0,-1.230924e+09,9.806323e+09,...,2.640000e+12,6867.930797,-8777.141881,-5053.114895,634.284660,-0.782479,-0.119529,-0.166111,-0.092263,-0.000347
2664,(주)엘지생활건강,51900,2013/12,1.0,607000.0,3.435104e+09,1.959727e+09,365652541.0,3.895183e+08,4.326255e+09,...,8.558772e+12,6425.644203,25926.923831,24338.380022,2191.328330,0.247837,0.106446,0.372706,0.667937,0.000055
16715,한올바이오파마(주),9420,2013/12,1.0,8670.0,9.785160e+07,3.822538e+07,-21498897.0,-7.780099e+06,7.422831e+07,...,1.743216e+11,6078.506501,-16858.465272,-46585.320889,4882.751459,-0.360561,-0.219709,-0.350634,0.324475,-0.000118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14790,태광산업(주),3240,2013/12,1.0,968000.0,3.994993e+09,1.291162e+09,113939554.0,2.338045e+08,3.513353e+09,...,1.436286e+12,398.608962,9459.148840,4609.711834,306.764261,0.042140,0.028521,0.077424,0.242960,0.000102
8781,동국제강(주),1230,2013/12,1.0,13050.0,9.109805e+09,6.490860e+09,-118408748.0,9.999798e+07,6.690855e+09,...,8.129894e+11,308.065629,-6813.744745,8068.233065,120.583536,-0.045212,-0.012998,-0.031364,0.047462,-0.000141
1204,(주)두산,150,2013/12,1.0,128500.0,3.116604e+10,2.210806e+10,130167997.0,3.833616e+08,2.193653e+10,...,2.913049e+12,296.241006,20614.484246,6999.517976,122.323158,0.014371,0.004177,0.010575,0.118020,0.000073
5690,(주)한화,880,2013/12,1.0,33450.0,1.133226e+11,1.025467e+11,224668731.0,6.404167e+09,3.872497e+10,...,2.938382e+12,232.681876,11160.296649,391.521616,64.748140,0.020849,0.001983,0.002218,0.024414,0.000133


In [316]:
kospi.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18345 entries, 0 to 18344
Data columns (total 29 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   회사명       18345 non-null  object 
 1   종목코드      18345 non-null  object 
 2   회계년도      18345 non-null  object 
 3   거래월       18345 non-null  float64
 4   종가        18345 non-null  float64
 5   자산        18345 non-null  float64
 6   부채        18345 non-null  float64
 7   당기순이익     18345 non-null  float64
 8   현금흐름      18345 non-null  float64
 9   매출액       18345 non-null  float64
 10  비유동자산     18345 non-null  float64
 11  매출총이익     18345 non-null  float64
 12  법인세비용     18345 non-null  float64
 13  이자비용      18345 non-null  float64
 14  감가상각비     18345 non-null  float64
 15  유동자산      18345 non-null  float64
 16  유동부채      18345 non-null  float64
 17  현금및현금성자산  18345 non-null  float64
 18  상장주식수     18345 non-null  float64
 19  시가총액      18345 non-null  float64
 20  pbr       18345 non-null  float64

In [317]:
df_array = []

for year in range(2013, 2023):
    for month in range(1, 13):
        fiscal_year = f"{year}/12"
        df = kospi.loc[(kospi["회계년도"] == fiscal_year) & (kospi["거래월"] == float(month))].sort_values(by=["pbr"], ascending=False)

        df_array.append(df)

df_array[119]


,회사명,종목코드,회계년도,거래월,종가,자산,부채,당기순이익,현금흐름,매출액,...,시가총액,pbr,per,pcr,psr,roe,roa,roic,gpa,매직
12802,에스케이바이오팜(주),326030,2022/12,12.0,72100.0,6.642411e+08,3.476807e+08,-1.394309e+08,-1.599201e+08,2.461791e+08,...,5.646385e+12,17836.672732,-40495.946135,-35307.548374,22936.085420,-0.440456,-0.209910,-0.285763,0.313758,-0.000025
5190,(주)하나투어,39130,2022/12,12.0,60500.0,4.554294e+08,3.557278e+08,-6.457242e+07,-1.024087e+07,1.149695e+08,...,9.703707e+11,9732.744861,-15027.632809,-94754.749501,8440.247238,-0.647657,-0.141784,-0.238680,0.252442,-0.000071
10736,삼성바이오로직스(주),207940,2022/12,12.0,821000.0,1.658205e+10,7.597576e+09,7.980562e+08,9.530478e+08,3.001295e+09,...,5.843385e+13,6503.870057,73220.223423,61312.615576,19469.545694,0.088826,0.048128,0.123583,0.088556,0.000017
6409,(주)호텔신라,8770,2022/12,12.0,83100.0,2.938545e+09,2.398761e+09,-5.016425e+07,2.204604e+08,4.922009e+09,...,3.261519e+12,6042.267209,-65016.794332,14794.123152,662.639795,-0.092934,-0.017071,-0.033922,1.674982,-0.000018
2183,(주)에프앤에프,383220,2022/12,12.0,144500.0,1.570179e+09,6.270067e+08,4.426645e+08,3.440717e+08,1.808895e+09,...,5.535372e+12,5868.887803,12504.666909,16087.847099,3060.085701,0.469336,0.281920,0.736296,0.812762,0.000109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12630,에스케이(주),34730,2022/12,12.0,189000.0,1.945059e+11,1.226971e+11,3.966176e+09,7.677749e+09,1.345516e+11,...,1.401422e+13,195.160275,3533.434518,1825.303638,104.154978,0.055232,0.020391,0.061024,0.077628,0.000449
6912,HD현대(주),267250,2022/12,12.0,57100.0,6.557086e+10,4.230720e+10,2.234955e+09,2.133778e+09,6.084967e+10,...,4.510505e+12,193.886275,2018.163734,2113.858515,74.125386,0.096071,0.034085,0.074175,0.090994,0.000637
11892,씨제이(주),1040,2022/12,12.0,84100.0,4.824041e+10,3.050677e+10,6.868457e+08,4.044986e+09,4.092488e+10,...,2.453786e+12,138.369023,3572.542665,606.623992,59.958283,0.038731,0.014238,0.045266,0.232767,0.000425
1323,(주)두산,150,2022/12,12.0,83100.0,2.631535e+10,1.601754e+10,-5.811687e+08,5.331835e+08,1.699576e+10,...,1.373131e+12,133.341936,-2362.705779,2575.343711,80.792544,-0.056436,-0.022085,-0.061007,0.107848,-0.000266


In [318]:
fiscal_year = f"{2013}/{1}"  
kospi.loc[(kospi["회계년도"] == fiscal_year)]

,회사명,종목코드,회계년도,거래월,종가,자산,부채,당기순이익,현금흐름,매출액,...,시가총액,pbr,per,pcr,psr,roe,roa,roic,gpa,매직


In [319]:
kospi["회계년도"][0].split('/')[0]


'2013'

In [320]:
float('1')

1.0

In [321]:
df

,회사명,종목코드,회계년도,거래월,종가,자산,부채,당기순이익,현금흐름,매출액,...,시가총액,pbr,per,pcr,psr,roe,roa,roic,gpa,매직
12802,에스케이바이오팜(주),326030,2022/12,12.0,72100.0,6.642411e+08,3.476807e+08,-1.394309e+08,-1.599201e+08,2.461791e+08,...,5.646385e+12,17836.672732,-40495.946135,-35307.548374,22936.085420,-0.440456,-0.209910,-0.285763,0.313758,-0.000025
5190,(주)하나투어,39130,2022/12,12.0,60500.0,4.554294e+08,3.557278e+08,-6.457242e+07,-1.024087e+07,1.149695e+08,...,9.703707e+11,9732.744861,-15027.632809,-94754.749501,8440.247238,-0.647657,-0.141784,-0.238680,0.252442,-0.000071
10736,삼성바이오로직스(주),207940,2022/12,12.0,821000.0,1.658205e+10,7.597576e+09,7.980562e+08,9.530478e+08,3.001295e+09,...,5.843385e+13,6503.870057,73220.223423,61312.615576,19469.545694,0.088826,0.048128,0.123583,0.088556,0.000017
6409,(주)호텔신라,8770,2022/12,12.0,83100.0,2.938545e+09,2.398761e+09,-5.016425e+07,2.204604e+08,4.922009e+09,...,3.261519e+12,6042.267209,-65016.794332,14794.123152,662.639795,-0.092934,-0.017071,-0.033922,1.674982,-0.000018
2183,(주)에프앤에프,383220,2022/12,12.0,144500.0,1.570179e+09,6.270067e+08,4.426645e+08,3.440717e+08,1.808895e+09,...,5.535372e+12,5868.887803,12504.666909,16087.847099,3060.085701,0.469336,0.281920,0.736296,0.812762,0.000109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12630,에스케이(주),34730,2022/12,12.0,189000.0,1.945059e+11,1.226971e+11,3.966176e+09,7.677749e+09,1.345516e+11,...,1.401422e+13,195.160275,3533.434518,1825.303638,104.154978,0.055232,0.020391,0.061024,0.077628,0.000449
6912,HD현대(주),267250,2022/12,12.0,57100.0,6.557086e+10,4.230720e+10,2.234955e+09,2.133778e+09,6.084967e+10,...,4.510505e+12,193.886275,2018.163734,2113.858515,74.125386,0.096071,0.034085,0.074175,0.090994,0.000637
11892,씨제이(주),1040,2022/12,12.0,84100.0,4.824041e+10,3.050677e+10,6.868457e+08,4.044986e+09,4.092488e+10,...,2.453786e+12,138.369023,3572.542665,606.623992,59.958283,0.038731,0.014238,0.045266,0.232767,0.000425
1323,(주)두산,150,2022/12,12.0,83100.0,2.631535e+10,1.601754e+10,-5.811687e+08,5.331835e+08,1.699576e+10,...,1.373131e+12,133.341936,-2362.705779,2575.343711,80.792544,-0.056436,-0.022085,-0.061007,0.107848,-0.000266
